In [3]:
#  Python code to read drive details (Drive letter, caapcity, Free space) from Windows server including Mountpoints
pip install wmi 

  Using cached WMI-1.5.1-py2.py3-none-any.whl.metadata (3.6 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import wmi
import pythoncom
import threading

def get_drive_details_wmi():
    """Get drive details using WMI"""
    pythoncom.CoInitialize()  # Initialize COM for this thread
    
    try:
        c = wmi.WMI()
        drive_details = []
        
        # Get logical disks
        for disk in c.Win32_LogicalDisk():
            if disk.DriveType == 3:  # Fixed drives
                try:
                    total_size = int(disk.Size) if disk.Size else 0
                    free_space = int(disk.FreeSpace) if disk.FreeSpace else 0
                    used_space = total_size - free_space
                    
                    drive_info = {
                        'drive_letter': disk.DeviceID,
                        'volume_name': disk.VolumeName or 'N/A',
                        'file_system': disk.FileSystem or 'N/A',
                        'total_size_gb': round(total_size / (1024**3), 2),
                        'free_space_gb': round(free_space / (1024**3), 2),
                        'used_space_gb': round(used_space / (1024**3), 2),
                        'free_space_percent': round((free_space / total_size) * 100, 2) if total_size > 0 else 0
                    }
                    
                    drive_details.append(drive_info)
                    
                except (ValueError, AttributeError):
                    continue
        
        return drive_details
        
    finally:
        pythoncom.CoUninitialize()

def print_drive_details_wmi(drive_details):
    """Print WMI drive details"""
    print(f"{'Drive':<6} {'Volume Name':<15} {'File System':<8} {'Total (GB)':<10} {'Free (GB)':<10} {'Used (GB)':<10} {'Free %':<8}")
    print("-" * 70)
    
    for drive in drive_details:
        print(f"{drive['drive_letter']:<6} {drive['volume_name']:<15} {drive['file_system']:<8} "
              f"{drive['total_size_gb']:<10} {drive['free_space_gb']:<10} {drive['used_space_gb']:<10} "
              f"{drive['free_space_percent']:<8}%")

if __name__ == "__main__":
    print("Retrieving drive details using WMI...")
    drives = get_drive_details_wmi()
    print_drive_details_wmi(drives)

Retrieving drive details using WMI...
Drive  Volume Name     File System Total (GB) Free (GB)  Used (GB)  Free %  
----------------------------------------------------------------------
C:     N/A             NTFS     237.33     10.43      226.9      4.4     %


Win32 exception occurred releasing IUnknown at 0x000001B73EDF7B60
Win32 exception occurred releasing IUnknown at 0x000001B73EE08920
Win32 exception occurred releasing IUnknown at 0x000001B73EE07F20
Win32 exception occurred releasing IUnknown at 0x000001B73EE091E0
Win32 exception occurred releasing IUnknown at 0x000001B73EE07E80
Win32 exception occurred releasing IUnknown at 0x000001B73EE07FC0
Win32 exception occurred releasing IUnknown at 0x000001B73EE08560


In [7]:
import wmi
import pythoncom

def get_drive_details():
    # Initialize COM before WMI calls
    pythoncom.CoInitialize()
    c = wmi.WMI()
    
    print("{:<15} {:<20} {:<20} {:<20}".format("Drive", "Capacity (GB)", "Free Space (GB)", "File System"))
    print("="*80)

    for disk in c.Win32_LogicalDisk(DriveType=3):  # DriveType=3 = Local Disk
        drive_letter = disk.DeviceID
        file_system = disk.FileSystem if disk.FileSystem else "Unknown"
        
        capacity_gb = int(disk.Size) / (1024**3) if disk.Size else 0
        free_space_gb = int(disk.FreeSpace) / (1024**3) if disk.FreeSpace else 0
        
        print("{:<15} {:<20.2f} {:<20.2f} {:<20}".format(
            drive_letter, capacity_gb, free_space_gb, file_system
        ))
    pythoncom.CoUninitialize()

def get_mountpoints():
    pythoncom.CoInitialize()
    c = wmi.WMI()
    print("\nMount Points:")
    print("="*80)
    for volume in c.Win32_Volume():
        if volume.DriveLetter is None and volume.Label:
            capacity_gb = int(volume.Capacity) / (1024**3) if volume.Capacity else 0
            free_space_gb = int(volume.FreeSpace) / (1024**3) if volume.FreeSpace else 0
            print(f"Label: {volume.Label}, Capacity: {capacity_gb:.2f} GB, Free: {free_space_gb:.2f} GB, Path: {volume.Name}")
    pythoncom.CoUninitialize()

if __name__ == "__main__":
    get_drive_details()
    get_mountpoints()


Drive           Capacity (GB)        Free Space (GB)      File System         
C:              237.33               10.43                NTFS                

Mount Points:
Label: System Reserved, Capacity: 0.57 GB, Free: 0.53 GB, Path: \\?\Volume{974cd22e-0000-0000-0000-100000000000}\
